In [ ]:
import requests

In [ ]:
# GET
TERM_URL = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/getTerms?offset={}&max={}"
SUBJECTS_URL = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/get_subject?searchTerm={}&term={}&offset={}&max={}"
SEARCH = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/searchResults/searchResults?txt_subject={}&txt_courseNumber={}&txt_term={}&pageOffset={}&pageMaxSize={}"

# POST
SET_TERM = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/term/search?mode=search"

In [ ]:
class RequestFailedException(Exception):
    pass

def get(url, **kwargs):
    print(f"fetching: {url}")
    response = requests.get(url, **kwargs)
    
    # 200 means good 
    if response.status_code == 200:
        return response
    else:
        raise RequestFailedException(f"Request failed with unknown status code: {response.status_code}")

def post(url, **kwargs):
    print(f"POST: {url}")
    response = requests.post(url, **kwargs)

    if response.status_code == 200:
        return response
    else:
        raise RequestFailedException(f"Request failed with unknown status code: {response.status_code}")

In [ ]:
page_offset = 1 #starts at 1
number_results = 10

QUERY_URL = TERM_URL.format(page_offset, number_results)
terms = get(QUERY_URL).json()

for term in terms:
    code, description = term.values()
    print(f"term_id:\t{code}\tdescription:\t'{description}'")

In [ ]:
# select code from previous cell
TERM_ID = 202430

In [ ]:
search_term = "Computer"
page = 1
num_results = 100

QUERY_URL = SUBJECTS_URL.format(search_term, TERM_ID, page, num_results)
results = get(QUERY_URL).json()

terms = results
for item in results:
    code, description = item.values()
    print(f"code:\t{code}\tdescription:\t{description}")

In [ ]:
# post to get cookies, if no cookies are given then they are returned and can be used later
PAYLOAD = {"term": TERM_ID
           }
HEADERS = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UT;",
}

response = post(SET_TERM, data=PAYLOAD, headers=HEADERS)
print(response.json())
COOKIES = response.cookies

response = post(SET_TERM, data=PAYLOAD, cookies=COOKIES, headers=HEADERS)

response.json()

In [ ]:
# Search
subject = "CS"
course_number = 2511
page = 0
num_results = 50

QUERY_URL = SEARCH.format(subject, course_number, TERM_ID, page, num_results)
print(QUERY_URL)

# have to get with the headers that we got before
response = requests.get(QUERY_URL, cookies=COOKIES)
search = response.json()

data = search["data"]
if data is None or len(data) == 0:
    print("No classes found")
else:
    for item in data:
        # if item["courseReferenceNumber"] != "34912":
        #     continue
        print(f"CRN: {item['courseReferenceNumber']}:\ttitle: {item['courseTitle']}")
        print(f"\tProfessor: {item['faculty'][0]['displayName']}")
        print(f"\tenrollment {item['enrollment']} / {item['maximumEnrollment']}")
        print(f"\twaitlist {item['waitCount']} / {item['waitCapacity']}")
        print(f"\ttime: {item['meetingsFaculty'][0]['meetingTime']['beginTime']} - {item['meetingsFaculty'][0]['meetingTime']['endTime']}")
        print(f"\tTuesday: {item['meetingsFaculty'][0]['meetingTime']['tuesday']}")

import json
with open(f"./data/classes.json", "w") as data_file:
    json.dump(data, data_file, indent=4)
    
# Reset the form for some reason banner requires this
if post("https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/resetDataForm", cookies=COOKIES).json():
    print("reset successful")
else:
    print("reset failed")